In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [2]:
df=pd.read_csv('energydata_complete.csv')
del df['date']
del df['lights']
df.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


### Linear Regression

In [3]:
#Normalising dataset using the min max scaler
MMscaler = MinMaxScaler()
normalising = pd.DataFrame(MMscaler.fit_transform(df), columns=df.columns)

#segmenting features & target variables
features = normalising.drop(columns=['T2', 'T6'])
target = normalising['T6']

#Train-Test Split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=1)

#Linear regression Modelling
LRmodel = LinearRegression()
LRmodel.fit(x_train, y_train)
predicted_values = LRmodel.predict(x_test)

### Solution 12

In [4]:
rsquared = r2_score(y_test, predicted_values)
round(rsquared,2)

0.97

### Multiple Linear Regression

In [5]:
#segmenting features & target variables
features = normalising.drop(columns=['Appliances'])
target = normalising['Appliances']

#Train-Test Split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

#Multiple Linear Regression Modelling
MLRmodel = LinearRegression()
MLRmodel.fit(x_train, y_train)
predicted_values = MLRmodel.predict(x_test)

### Solution 13

In [6]:
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2)

0.05

### Solution 14

In [7]:
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

### Solution 15

In [8]:
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

### Solution 16

In [9]:
rsquared = r2_score(y_test, predicted_values)
round(rsquared, 2)

0.15

### Solution 17

In [10]:
a = LRmodel.coef_
np.min(a) #RH_2
np.max(a) #RH_1

1.0336401347441457

### Solution 18 - Ridge Regression

In [11]:
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
predicted_values_ridge = ridge_reg.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, predicted_values_ridge))
round(rmse, 3)

0.088

### Lasso Regression

In [12]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

#this function returns the weight of every feature
def get_weights_df(model, feat, col_name): 
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
pd.DataFrame(lasso_weights_df)

,Features,Lasso_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


### Solution 19

In [13]:
a = len(lasso_weights_df[lasso_weights_df['Lasso_weight'] == 0])
len(lasso_weights_df) - a #Non Zeros

4

### Solution 20

In [14]:
predicted_values_lasso = lasso_reg.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted_values_lasso))
round(rmse, 3)

0.094